# Data Preparation and Exploration

## Setup

In [1]:
#import libraries
import pandas as pd
import numpy as np
import janitor 

First, I will read in my Parquet files.

I will clean the variable names to make them uniform and join the two datasets together.

Then I will remove true duplicates, as each instance should have a unique identifier; according to the metadata (City of Los Angeles, 2025a; 2025b), this is `dr_no` - Division of Records number. Any row that is a complete duplicate is therefore likely to be attributable to a data entry error. 

In [2]:
#get 2010-19 data from parquet
df1 = pd.read_parquet("../data/la_crimes_2010-19.parquet")

#get 2020-25 data from parquet
df2 = pd.read_parquet("../data/la_crimes_2020-25.parquet")

#clean variable names
df1 = (
    df1.clean_names()
    .rename(columns={"date_occ":"date", "time_occ":"time_str", "area_name":"area", "crm_cd":"crime_code", "crm_cd_desc":"crime_type", "premis_cd":"premises_code", "premis_desc":"premises_type", "weapon_used_cd":"weapon_code", "weapon_desc":"weapon_type"})
    )

df2 = (
    df2.clean_names()
    .rename(columns={"date_occ":"date", "time_occ":"time_str", "area":"area_", "area_name":"area", "crm_cd":"crime_code", "crm_cd_desc":"crime_type", "premis_cd":"premises_code", "premis_desc":"premises_type", "weapon_used_cd":"weapon_code", "weapon_desc":"weapon_type"})
    )

#join dataframes and view all columns
df = pd.concat([df1, df2], ignore_index=True)
pd.set_option('display.max_columns', None)
df.head(4)

,dr_no,date_rptd,date,time_str,area_,area,rpt_dist_no,part_1_2,crime_code,crime_type,mocodes,vict_age,vict_sex,vict_descent,premises_code,premises_type,weapon_code,weapon_type,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,None,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,None,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,0045,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,0,M,W,101.0,STREET,NaN,None,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,0,M,H,103.0,ALLEY,NaN,None,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,None,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,0150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,1100 0400 1402,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295


In [3]:
#remove duplicate rows and check size
df = df.drop_duplicates()
df.shape

(3078018, 28)

## Exploring and Cleaning Variables

### Demographics

I will view the unique values that appear in the demographic variables to check whether they need cleaning, using the metadata to support my understanding and decisions.

In [4]:
#get demographic values and counts
vict_sex_values = df['vict_sex'].value_counts(dropna=False).to_dict()
vict_descent_values = df['vict_descent'].value_counts(dropna=False).to_dict()

print(f"Victim Sex Values:\n{vict_sex_values}\n\nVictim Descent Values:\n{vict_descent_values}")

Victim Sex Values:
{'M': 1359082, 'F': 1229587, None: 337219, 'X': 151926, 'H': 185, 'N': 17, '-': 2}

Victim Descent Values:
{'H': 1006084, 'W': 704253, 'B': 464311, None: 337277, 'O': 276859, 'X': 183028, 'A': 71246, 'K': 14333, 'F': 7368, 'C': 5693, 'J': 1999, 'I': 1959, 'V': 1401, 'Z': 717, 'P': 624, 'U': 408, 'G': 152, 'D': 115, 'L': 97, 'S': 89, '-': 5}


In [5]:
#tidy victim sex variable
vict_sex_map = {
    "M": "Male",
    "F": "Female",
    "X": "Other/Unknown",
    "H": "Other/Unknown",
    "N": "Other/Unknown",
    "-": "Other/Unknown"
}
df["vict_sex"] = df["vict_sex"].map(vict_sex_map).fillna("Other/Unknown")

#tidy victim descent variable
vict_descent_map = {
    "A": "Other Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian",
    "-": "Unknown"
}
df["vict_descent"] = df["vict_descent"].map(vict_descent_map).fillna("Unknown")

### Dates and Times

I will clean the date and time variables by converting them into datetime objects. This will allow me to extract granular features later, including day of the week and hour of the day.


In [6]:
#convert dates
df["date"] = pd.to_datetime(df["date"], format="%m/%d/%Y %I:%M:%S %p").dt.normalize()

#convert times
df["time"] = pd.to_datetime(df["time_str"], format="%H%M", errors="coerce").dt.time

#get datetime column
df["datetime_str"] = df["date"].dt.strftime("%Y-%m-%d") + " " + df["time_str"].str[:2] + ":" + df["time_str"].str[2:]
df["datetime"] = pd.to_datetime(df["datetime_str"], format="%Y-%m-%d %H:%M")
df.drop(columns="datetime_str", inplace=True)

### Removing Unnecessary Data

I will drop rows with missing victim age, as this variable is essential for building my model. I will also drop rows where the age is zero or less (vict_age contains many 0s and negative numbers, possibly as crimes without known/human victims e.g. vandalism). Also, I've identified that there are three observations where `vict_age` is unusually high (114, 118, 120). These aren't necessarily entirely impossible as far as ages go, but I think they are unlikely enough that I will drop them in case they are errors. This is further justified as all three have 'Unknown' for `vict_sex` which makes me think that they were related to a non-human victim or somehow logged incorrectly.

In [7]:
#drop columns that won't be used for the model
df = df.drop(columns=["dr_no", "date_rptd", "area_", "rpt_dist_no", "part_1_2", "crime_code", "mocodes", "premises_code", "weapon_code", "status", "status_desc", "crm_cd_1", "crm_cd_2", "crm_cd_3", "crm_cd_4", "location", "cross_street"])

#drop rows with missing victim age
df = df.dropna(subset=["vict_age"])

#get realistic age range (1 to 99)
df = df[df["vict_age"].between(1, 99)]

In [8]:
df.head(4)

,date,time_str,area,crime_type,vict_age,vict_sex,vict_descent,premises_type,weapon_type,lat,lon,time,datetime
0,2010-02-20,1350,Newton,VIOLATION OF COURT ORDER,48,Male,Hispanic/Latin/Mexican,SINGLE FAMILY DWELLING,None,33.9825,-118.2695,13:50:00,2010-02-20 13:50:00
3,2010-01-05,0150,Hollywood,VIOLATION OF COURT ORDER,47,Female,White,STREET,HAND GUN,34.1016,-118.3295,01:50:00,2010-01-05 01:50:00
4,2010-01-02,2100,Central,"RAPE, ATTEMPTED",47,Female,Hispanic/Latin/Mexican,ALLEY,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",34.0387,-118.2488,21:00:00,2010-01-02 21:00:00
5,2010-01-04,1650,Central,SHOPLIFTING - PETTY THEFT ($950 & UNDER),23,Male,Black,DEPARTMENT STORE,None,34.0480,-118.2577,16:50:00,2010-01-04 16:50:00


## Dataframe Export

In [9]:
# df.to_csv("../data/df_for_grouping.csv", index=False, encoding="utf-8")